# Welcome

This is the haiopy demo and a good place for getting started. In this notebook, you will see examples of the most important haiopy functionalty.

**Note:** This is not a substitute for the haiopy documentaion at **insert_url**.

## Contents
[Signals](#signals)
- [Signal types](#signal_types)
- [FFT normalization](#fft_normalization) TODO
- [Accessing Signal data](#accessing_signal_data)
- [Iterating Signals](#accessing_signal_data)
- [Signal meta data](#signal_meta_data) TODO
- [Arithmetic operations](#arithmetic_operations) TODO

[Coordinates](#coordinates)
- [Entering coordinate points](#coordinates_enter)
- [Retrieving coordinate points](#coordinates_retrieve)
- [Rotating coordinate points](#coordinates_rotate)

[Orientations](#orientations)
- [Entering orientations](#entering_orientations)
- [Retrieving orientations](#retrieving_orientations)
- [Rotating orientations](#rotating_orientations)

[Plotting](#plotting) TODO

[DSP] TODO
- Filtering TODO

[In'n'out](#in_and_out)
- [Read and write workspace](#io_workspaces) TODO
- [Read and write wav files](#io_wav_files)
- [Read SOFA files](#io_sofa) TODO


Lets start with importing haiopy and numpy:

In [ ]:
# import packages
import haiopy
from haiopy import Signal                     # managing audio signals
from haiopy.coordinates import Coordinates    # managing satial sampling points
from haiopy.orientations import Orientations  # managing orientation vectors
from haiopy.spatial import samplings          # creating predefined spatial sampling grids
import numpy as np                            # you gotta love numpy, but pandas are cuter

# Signals<a class="anchor" id="signals"></a>

Audio signals are the basis of haiopy. They are stored in objects of the `Signal` class along with information about the sampling rate, the domain (`time`, or `freq`), the FFT type and an optional comment. Lets go ahead and create a single channel signal:

In [ ]:
# create a dirac signal with a sampling rate of 44.1 kHz
fs = 44100
x = np.zeros(441)
x[0] = 1
x_energy = Signal(x, fs, signal_type='energy')

# show information
x_energy

## Signal types<a class="anchor" id="signal_types"></a>

There are two different *signal types*: `'energy'` and `'power'`. Energy signals are finite signals with a finite energy. The most common example for an energy signal is an impulse response. Power signals are finite sections of infinte audio signals. They have infinite energy but finite power. Examples for power signals are environmental noise or sine tones. It is important to distinguish between the two and some functions have signal type specific behaviour. You will come acros this in the sections [FFT Normalization](#fft_normalization), [Arithmetic operations](#arithmetic_operations), and [Plotting](#plotting). Lets also create a power signal

In [ ]:
x = np.sin(2 * np.pi * 1000 * np.arange(441) / fs)
x_power = Signal(x, fs, signal_type='power')

## FFT Normalization<a class="anchor" id="fft_normalization"></a> [ToDo]

## Accessing Signal data<a class="accessing_signal_data" id="signals"></a>

You can access the data, i.e., the audio signal, inside a Signal object in the time and frequency domain by simply using

In [ ]:
time_data = x_power.time
freq_data = x.power.freq

Please note that the frequency data of `'power'` signals depends on the Signal's `fft_norm`. Internally, the data are stored either in the `'time'` or `'freq'` domain. It is tranformed every time you access it without changing the internal state. In some cases it might be more efficient to store the data in a specif domain to avoid additional Fourier Transforms. Lets check out the current domain

In [ ]:
x_power.domain

and change it

In [ ]:
x_power.domain = 'freq'
x_power.domain

## Iterating Signals<a class="signal_meta_data" id="accessing_signal_data"></a>

It is the aim of pyfar that all operations work on N-dimensional `signals`. Nevertheless, you can also iterate `signals` if you need to apply operations depending on the channel. Lets look at a simple example

In [ ]:
signal = Signal([[0, 0, 0], [1, 1, 1]], 44100)  # 2-channel signal

# iterate the signal
for n, channel in enumerate(signal):
    print(f"Channel: {n}, time data: {channel.time}")
    # do something channel dependent
    channel.time = channel.time + n
    # write changes to the signal
    signal[n] = channel

# q.e.d.
print(f"\nNew signal time data:\n{signal.time}")

`Signal` uses the standard `numpy` iterator which always iterates the first dimension. In case of a 2-D array as in the example above these are the channels.

## Signal meta data<a class="signal_meta_data" id="signals"></a> ([pick the most important])

- cshape, etc
- n_samples
- time
- comment ...

## Arithmetic operations<a class="arithmetic_operations" id="signals"></a> [ToDo]


# Plotting<a class="anchor" id="plotting"></a> [TODO]

# Coordinates<a class="anchor" id="coordinates"></a>

The `Coordinates()` class is designed for storing, manipulating, and acessing coordinate points in a large variety of different coordinate conventions. Examples for data that can be stored are microphone positions of a spherical microphone array and loudspeaker positions of a sound field synthesis system. Lets create and empty `Coordinates` object and look at the implemented conventions first:

In [ ]:
c = Coordinates()
c.systems()

## Entering coordinate points<a class="anchor" id="coordinates_enter"></a>

Coordinate points can be entered manually or by using one of the available sampling schemes contained in `haiopy.spatial.samplings`. We will do the latter using an equal angle sampling and look at the information provided by the coordinates object:

In [ ]:
c = samplings.sph_equal_angle((20, 10))
# show general information
print(c)
# plot the sampling points
c.show()

Inside the `Coordinates` object, the points are stored in an N-dimensional array of size `[..., 3]` where the last dimension in this case holds the azimuth, colatitude, and radius. Information about coordinate array can be obtained by `c.cshape`, `c.csize`, and `c.cdim`. These properties are similar to numpy's `shape`, `size`, and `dim` but ignore the last dimension, which is always 3.

## Retrieving coordinate points<a class="anchor" id="coordinates_retrieve"></a>

There are different ways to retrieve points from a `Coordinates` object. All points can be obtained in cartesian, spherical, and cylindrical coordinates using the getter functions `c.get_cart()`, `c.get_sph()` and `c.get_cyl()`, e.g.:

In [ ]:
cartesian_coordinates = c.get_cart()

Different methods are available for obtaining a specific subset of coordinates. For example the nearest point(s) can be obtained by

In [ ]:
c_out = c.get_nearest_k(
    270, 90, 1, k=1, domain='sph', convention='top_colat', unit='deg', show=True)

To obtain all points within a specified eucledian distance or arc distance, you can use `c.get_nearest_cart()` and `c.get_nearest_sph()`. To obtain more complicated subsets of any coordinate, e.g., the horizontal plane with `colatitude=90` degree, you can use slicing

In [ ]:
mask_hor = c.get_slice('colatitude', 'deg', 90, show=True)

## Rotating coordinates<a class="anchor" id="coordinates_rotate"></a>

You can apply rotations using quaternions, rotation vectors/matrixes and euler angles with  `c.rotate()`, which is a wrapper for `scipy.spatial.transform.Rotation`. For example rotating around the y-axis by 45 degrees can be done with

In [ ]:
c.rotate('y', 45)
c.show()

Note that this changes the points inside the `Coordinates` object, which means that you have to be carefull not to apply the rotation multiple times, i.e., when evaluationg cells during debugging.

# Orientations<a class="anchor" id="orientations"></a>

The `Orientations()` class is designed storing, manipulating, and accessing orientation vectors. Examples for this are the orientations of directional loudspeakers when measuring room impulse responses or the head orientation belonging to binaural impulse responses. It is good to know that `Orientations` is inherited from `scipy.spatial.transform.Rotation` and that all methods of this class can also be used with `Orientations`.

## Entering orientations<a class="anchor" id="entering_orientations"></a>

Lets go ahead and create an object and show the result

In [ ]:
views = [[0,  1, 0],
         [1,  0, 0],
         [0, -1, 0]]
up = [0, 0, 1]
orientations = Orientations.from_view_up(views, up)
orientations.show(show_rights=False)


It is also possible to enter `Orientations` from `Coordinates` object or mixtures of `Coordinates` objects and array likes. This is equivalent to the example above

In [ ]:
views_c = Coordinates([90, 0, 270], 0, 1,
                      domain='sph', convention='top_elev', unit='deg')

orientations = Orientations.from_view_up(views_c, up)

## Retrieving orientations<a class="anchor" id="retrieving_orientations"></a>

Orientaions can be retrieved as view, up, and right-vectors and in any format supported by `scipy.spatial.transform.Rotation`. They can also easily converted into any coordinate convention supported by haiopy by putting them into a `Coordinates` object. Lets only check out one way for now 

In [ ]:
views, ups, right, = orientations.as_view_up_right()

In this case the output is identical to the input. This would not be the case if for exaple using `view = [2, 0, 0]` in which case the output would be the unit vector `[1, 0, 0]`.

## Rotating orientations<a class="anchor" id="rotating_orientations"></a>

Rotations can be done using the methods inherited from `scipy.spatial.transform.Rotation`. You can for example rotate around the y-axis this way

In [ ]:
rotation = Orientations.from_euler('y', 30, degrees=True)
orientations_rot = orientations * rotation
orientations_rot.show(show_rights=False)

# In'n'out<a class="in_and_out" id="signals"></a>

Now that you know what pyfar is about, let's see how you can save your work and read comman data types.

## Read and write workspace<a class="in_and_out" id="#io_workspaces"></a> [ToDo]

## Read and write wav-files<a class="in_and_out" id="wav_files"></a>

Wav-files are commonly used in the audio community to store and exchange data. You can read them with

`signal = haiopy.io.read_wav(filename)`

and write them with

`haiopy.io.write_wav(signal, filename, overwrite=True)`.

You can write any `signal` to a wav-file also if they have values > 1. Multidimensional `signals` will be reshaped to 2D arrays before writing.

## Read SOFA files<a class="in_and_out" id="#io_sofa"></a> [TODO]